In [10]:
#normalize any vector to the N sphere
function normalization(learn::Array{Float64,1})
    norm=0
    for c in learn
        norm=norm+c^2
    end
    learn_norm=sqrt(N)*learn/sqrt(norm)
    return(learn_norm)
end

#generate the underlying couplings with s=1; generating M sets of biased inputs and outputs 
function teacher()
    coupling=rand(d,N)   #draw from d, the standard normal distribution
    mag=sum(coupling)
    norm=sum(coupling.^2)
    while (mag>10.1)||(mag<9.9)||(norm>100.1)||(norm<99.9)
        coupling=rand(d,N)
        mag=sum(coupling)
        norm=sum(coupling.^2)
    end
    println(mag/10.0)  #print weak magnetisation of the teacher
    println(norm)   #print squared norm of the teacher
    
    observation=Array{Int64}(N+1,M)
    for i=1:M
        input=rand(N)
        for j=1:N
            if input[j]<0.5+m/2.0
                observation[j,i]=1
            else
                observation[j,i]=-1
            end
        end
        if dot(observation[1:N,i],coupling)>0
            output=1
        else
            output=-1
        end
        observation[N+1,i]=output
    end
    coupling,observation
end

#cost function with \rho(k)=-k
function myfunc(J::Array{Float64,1})
    student=J
    add=zeros(N)
    for c=1:M
        field=teach[2][N+1,c]*dot(teach[2][1:N,c],student)/sqrt(N)
        add=add-teach[2][1:N,c].*teach[2][N+1,c]./sqrt(N)   
    end
    student=student-add
    return(student)      
end

#evaluate the error between teacher and student
function error(J::Array{Float64,1})
    error=0.0
    for c=1:N
        error=error+(teach[1][c]-J[c])^2
    end
    error/N
end


m=0.4
N=100   #number of nodes
M=300   #number of observations
using Distributions
srand(123)   #setting seed
d=Normal()   #define d as the standard normal distribution
trial=10   #number of trials
data1=zeros(trial)
data2=zeros(trial)
teach=teacher()
for i=1:trial
    teach=teacher()
    initial=rand(d,N)
    student=initial
    for j=1:500
        student=myfunc(student)
    end
    student=normalization(student)
    data1[i]=error(student)
    data2[i]=sum(student)/10
end
average1=0.0
average2=0.0
for i=1:trial
    average1=average1+data1[i]
    average2=average2+data2[i]
end
average1=average1/trial
average2=average2/trial

standard1=0.0
standard2=0.0
for i=1:trial
    standard1=standard1+(data1[i]-average1)^2
    standard2=standard2+(data2[i]-average2)^2
end
standard1=sqrt(standard1/trial)
standard2=sqrt(standard2/trial)

println(average1)  #average of error between student and teacher
println(standard1)  #standard deviation of error between student and teacher
println(average2)  #average of weak magnetisation of students
println(standard2)  #standard deviation of weak magnetisation of students

0.9908790483481015
100.02693397648247
0.

9989000581162324
99.15707471992421
1.0003408025525284
99.93449278912242
1.0084315546922744
100.07462383587757
1.0080669908538569
99.95449882544403
0.9958689806067348
99.97850594012003
1.844423585920891
99.94683629014555
1.8727455456869265
100.08712121563138
0.9997046668817475
99.92511089933703
1.009517176905765
99.91716936966706
0.9925447112620052
99.92033860200053
1.003785634004051
0.09161621978791358
8.393183809624828
0.4405118771965961


In [61]:
#theoretical calculation
using SymPy
x=Sym("x")
k=Integral(x*exp(-x^2/2.0)/sqrt(2*pi),(x,0.4*0.3,50.0))
g=2*sqrt(3)*evalf(k)
2-2*sqrt(g^2/(1+g^2)) #error

0.383725923110590